In [ ]:
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm

from sklearn.metrics import mean_absolute_percentage_error as mean_ape
from sklearn.metrics import root_mean_squared_error as root_mse

In [ ]:
model_data_no_journal = pd.read_parquet("data/model_data_no_journal.parquet")
model_data_no_journal = model_data_no_journal[model_data_no_journal['political_alignment'] != 'autre']
model_data_no_journal['pres_votes_share'] = model_data_no_journal['pres_dummy'] * model_data_no_journal['pres_votes_share']

model_data = pd.read_parquet("data/model_data.parquet")
model_data = model_data[model_data['political_alignment'] != 'autre']
model_data['pres_votes_share'] = model_data['pres_dummy'] * model_data['pres_votes_share']

In [ ]:
nuances_order = [
    "Far right",
    "Right",
    "Center",
    "Left",
    "Far left"]

In [ ]:
outcome = "quotes_share"

Ce notebook utilise une approche économétrique classique cherchant à expliquer des phénomènes réels.
- Les outcomes sont la proportion de citations issues de chaque nuance politique
- Les régresseurs sont la proportion de sièges à l'Assemblée Nationale, la proportion des voix obtenues aux élections législatives et présidentielles, ainsi que la participation au gouvernement
- Les contrôles sont les périodes et les journaux

Les contrôles sont utilisés en interaction complète avec les régresseurs. Formellement, ils indicent les outcomes et les régresseurs, au lieu d'apparaître comme des variables distinctes. Cela revient à estimer des modèles distincts pour chaque sous-ensemble de données défini par une combinaison de nuance politique et de contrôles. Cette approche semble préférable pour expliquer la couverture médiatique de chaque nuance politique, dans chaque journal, à chaque période. Elle permet aussi d'éviter les problèmes de colinéarité.

Il serait possible d'utiliser les nuances politiques comme contrôles. Mais les variables correspondantes tendent à capturer l'essentiel des effets, réduisant l'intérêt sociologique des modèles.

Ces derniers sont par ailleurs dépourvus de constantes, dont l'ajout aboutit le plus souvent à dégrager les performances des modèles, tout en compliquant leur interprétation (par exemple avec une constante très élevée et un coefficient négatif pour la proportion de sièges à l'Assemblée Nationale).

Comme les modèles ne concernent pas toujours les mêmes populations et sont dépourvus de constante, le $R^2$ ne constitue pas une mesure de performance satisfaisante ([Shalizi, 2004](https://www.stat.cmu.edu/~cshalizi/TALR/TALR.pdf)). On utilise à la place deux grandeurs reflétant la distance relative entre les valeurs réelles et prédites :
- $\text{Mean absolute percentage error (MAPE)} = \frac{1}{n} \sum_{i=1}^{n} \left| \frac{\hat{y}_i - y_i}{y_i} \right| \Rightarrow$ Elle s'interprète directement : une MAPE de 1 signifie que les valeurs prédites s'écartent en moyenne de 100 % des valeurs réelles.
- $\text{Root mean squared percentage error (RMSPE)} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} \left(\frac{\hat{y}_i - y_i}{y_i}\right)^2} \Rightarrow$ Elle ne s'interprète pas directement mais permet aussi de comparer les modèles, en étant plus sensible aux outliers que la MAPE.

*NB 1: la relativité des écarts entre valeurs réelles et prédites est essentielle : un écart de 5 points de pourcentage n'a pas la même signification lorsque les valeurs réelles gravitent autour de 50 % ou de 5 % par exemple.*

*NB 2 : les données brutes sont des citations datées. Afin de les exploiter, il est nécessaire de les périodiser — ici, en les agrégant en nombre de citations par mois. De surcroît, les outcomes sont relatifs, prenant la forme de proportions de citations mensuelles. Dans ce contexte, pour que l’écart entre les valeurs observées et les valeurs prédites reflète correctement la performance d'un modèle, il est indispensable que son niveau d'agrégation corresponde à celui des données. Par exemple, si l’on estime un modèle unique pour l’ensemble des journaux, ces derniers ne doivent pas apparaître dans les données : on doit connaître la proportion de citations de personnalités du  pour chaque période, mais pas dans chaque journal à chaque période. Dans le cas contraire, la MAPE ou la RMSPE risquent d’être biaisées, car elles seraient calculées en moyenne sur des journaux ayant des volumes très différents d’articles.*

## 1. Modèle de base = sièges à l'AN
De la forme $Y_{i} = \beta T_{i}$ avec :
- $Y_{i}$ la proportion des articles intégrant des citations de la nuance politique $i$ (avec $\sum_{i=1}^{n} Y_{i} = 1$)
- $\beta$ la pondération du nombre de sièges détenus les différentes nuances politiques représentées à l'Assemblée
- $T_{i}$ la proportion des sièges détenus par la nuance politique $i$

In [ ]:
X = model_data_no_journal["na_share"]
y = model_data_no_journal[outcome]
model = sm.OLS(y, X).fit(cov_type='HC3')
y_pred = model.predict(X)
mape = mean_ape(y, y_pred)
rmspe = root_mse(y, y_pred) / y.mean()

params = model.params.rename("coef").to_frame()
pvalues = model.pvalues.rename("pval").to_frame()
print(pd.merge(params, pvalues, left_index=True, right_index=True))
print("")
print(f"MAPE: {100*mape:.2f}%")
print(f"RMSPE: {rmspe:.5f}")

## 2. Sièges à l'AN + participation au gouvernement
De la forme $Y_{i} = \beta T_{i} + \gamma G_{i}$ avec :
- $\gamma$ la proportion des articles intégrant des citations dont bénéficie la nuance politique du gouvernement
- $G_{i}$ une indicatrice valant 1 si le Premier ministre appartient à la nuance politique $i$

In [ ]:
X = model_data_no_journal[["na_share", "government"]]
y = model_data_no_journal[outcome]
model = sm.OLS(y, X).fit(cov_type='HC3')
y_pred = model.predict(X)
mape = mean_ape(y, y_pred)
rmspe = root_mse(y, y_pred) / y.mean()

params = model.params.rename("coef").to_frame()
pvalues = model.pvalues.rename("pval").to_frame()
print(pd.merge(params, pvalues, left_index=True, right_index=True))
print("")
print(f"MAPE: {100*mape:.2f}%")
print(f"RMSPE: {rmspe:.5f}")

## 3. Sièges à l'AN + participation au gouvernement + score aux présidentielles et aux législatives
Les modèles estimés sont de la forme $Y_{i} = \beta T_{i} + \gamma G_{i} + \theta P_{i} + \delta L_{i}$ avec :
- $\theta$ la pondération des résultats aux élections présidentielles des différentes nuances politiques
- $P_{i}$ la proportion des voix obtenues par la nuance politique $i$ à l'élection présidentielle suivante (pour le mois de l'élection et les trois précédents)
- $\delta$ la pondération des résultats aux élections législatives des différentes nuances politiques
- $L_{i}$ la proportion des voix obtenues par la nuance politique $i$ à l'élection législative précédente

In [ ]:
X = model_data_no_journal[["na_share", "government", "pres_votes_share", "leg_votes_share"]]
y = model_data_no_journal[outcome]
model = sm.OLS(y, X).fit(cov_type='HC3')
y_pred = model.predict(X)
mape = mean_ape(y, y_pred)
rmspe = root_mse(y, y_pred) / y.mean()

params = model.params.rename("coef").to_frame()
pvalues = model.pvalues.rename("pval").to_frame()
print(pd.merge(params, pvalues, left_index=True, right_index=True))
print("")
print(f"MAPE: {100*mape:.2f}%")
print(f"RMSPE: {rmspe:.5f}")

## 4. Sièges à l'AN + participation au gouvernement + score aux présidentielles et aux législatives // nuances politiques
Les modèles estimés sont de la forme $Y_{i} = \beta_{i} T_{i} + \gamma_{i} G_{i} + \theta_{i} P_{i} + \delta_{i} L_{i}$ : les effets deviennent spécifiques à chaque nuance politique.

In [ ]:
models = {}

for alignment in model_data_no_journal["political_alignment"].unique():
    subset = model_data_no_journal[model_data_no_journal["political_alignment"] == alignment]

    if len(subset) >= 3:
        X = subset[["na_share", "government", "pres_votes_share", "leg_votes_share"]]
        y = subset[outcome]
        model = sm.OLS(y, X).fit(cov_type='HC3')      
        y_pred = model.predict(X)
        mape = mean_ape(y, y_pred)
        rmspe = root_mse(y, y_pred) / y.mean()
        
        models[alignment] = {
            "params": model.params.rename("params").to_frame(),
            "pvalues": model.pvalues.rename("pval").to_frame(),
            "mape": mape,
            "rmspe": rmspe}

    else:
        models[alignment] = {
            "params": None,
            "pvalues": None,
            "mape": None,
            "rmspe": None}

In [ ]:
summary = []

for alignment, model_info in models.items():
    summary.append({
        "alignment": alignment,
        "mape": model_info["mape"],
        "rmspe": model_info["rmspe"]
    })

results = pd.DataFrame(summary)
results["alignment"] = pd.Categorical(results["alignment"], categories=nuances_order, ordered=True)
results = results.sort_values("alignment").reset_index(drop=True)

results.style.hide(axis="index") 

In [ ]:
results_list = []

for alignment, model_info in models.items():
    df = model_info["params"].join(model_info["pvalues"])
    df["alignment"] = alignment
    df["variable"] = df.index
    results_list.append(df.reset_index(drop=True))

results = pd.concat(results_list, ignore_index=True)
results = results.dropna(subset=["pval"])
results["alignment"] = pd.Categorical(results["alignment"], categories=nuances_order, ordered=True)
results = results.sort_values(["variable", "alignment"])
results = results.set_index(["variable", "alignment"])
results.index.names = ["Variable", "alignment"]

results.style.format("{:.4f}")

## 5. Sièges à l'AN + participation au gouvernement + score aux présidentielles et aux législatives // nuances politiques + périodes
Il s'agit maintenant d'étudier plus directement l'hypothèse d'une légitimation de l'extrême droite par la presse écrite nationale. La méthode la plus simple est d'estimer les modèles précédents pour 2 périodes, afin d'observer l'évolution des coefficients. Les modèles estimés sont de la forme $Y_{it} = \beta_{it} T_{it} + \gamma_{it} G_{it} + \theta_{it} P_{it} + \delta_{it} L_{it}$ : les effets deviennent spécifiques à chaque période $t$.

On retient 2017 comme charnière, avec l'idée que...
- La période précédente est encore marquée par le jeu fonctionnement traditionnel du système politique, avec une domination persistante de la droite et de la gauche de gouvernement, malgré leurs recompositions.
- La période suivante est caractérisée par l'effondrement de ce système, avec l'essort du centre et des extrêmes.

In [ ]:
cutoff1 = pd.Period('2002-06', freq='M')
cutoff2 = pd.Period('2017-06', freq='M')

models = {}

for period_label, period_filter in {
    '1981-2002': model_data_no_journal["month"] < cutoff1,
    '2002-2017': (model_data_no_journal["month"] >= cutoff1) & (model_data_no_journal["month"] < cutoff2),
    '2017-2024': model_data_no_journal["month"] >= cutoff2
}.items():
    
    models[period_label] = {}
    period_data = model_data_no_journal[period_filter]
    
    for alignment in period_data["political_alignment"].unique():
        models[period_label][alignment] = {}
        subset = period_data[period_data["political_alignment"] == alignment]

        if len(subset) >= 3:
            X = subset[["na_share", "government", "pres_votes_share", "leg_votes_share"]]
            y = subset[outcome]            
            model = sm.OLS(y, X).fit(cov_type='HC3')
            y_pred = model.predict(X)
            mape = mean_ape(y, y_pred)
            rmspe = root_mse(y, y_pred) / y.mean()

            models[period_label][alignment] = {
                "params": model.params.rename("params").to_frame(),
                "pvalues": model.pvalues.rename("pval").to_frame(),
                "mape": mape,
                "rmspe": rmspe}

        else:
            models[period_label][alignment] = {
                "params": None,
                "pvalues": None,
                "mape": None,
                "rmspe": None}

In [ ]:
summary_list = []

for period, alignment_dict in models.items():
    for alignment, model_info in alignment_dict.items():
        summary_list.append({
            "period": period,
            "alignment": alignment,
            "mape": model_info["mape"],
            "rmspe": model_info["rmspe"]
        })

summary = pd.DataFrame(summary_list)
summary["alignment"] = pd.Categorical(summary["alignment"], categories=nuances_order, ordered=True)
summary = summary.pivot_table(index="alignment",
                                      columns="period",
                                      values=["mape", "rmspe"],
                                      observed=False)
summary = summary.reset_index()
summary.columns.names = [None, None]

format_cols = summary.columns[1:]
summary.style.hide(axis="index").format({col: "{:.3f}" for col in format_cols})

In [ ]:
results_list = []

for period, alignment_dict in models.items():
    for alignment, model_info in alignment_dict.items():
        df = model_info["params"].join(model_info["pvalues"])
        df["period"] = period
        df["alignment"] = alignment
        df["variable"] = df.index
        results_list.append(df.reset_index(drop=True))

results = pd.concat(results_list, ignore_index=True)
results = results.dropna(subset=["pval"])
results["alignment"] = pd.Categorical(results["alignment"], categories=nuances_order, ordered=True)
results = results.pivot_table(index=["alignment", "variable"],
                              columns="period",
                              values=["params", "pval"],
                              observed=False)
results.columns = results.columns.swaplevel(0, 1)
results = results.reindex(columns=["1981-2002", "2002-2017", "2017-2024"], level=0)
results = results.sort_index()
results.index.names = ["alignment", "Variable"]

results.style.format({col: "{:.4f}" for col in results.columns})

## 6. Sièges à l'AN + participation au gouvernement + score aux présidentielles et aux législatives // nuances politiques + périodes + journaux
On différencie finalement selon les journaux. Les modèles estimés sont de la forme $Y_{ijt} = \beta_{ijt} T_{ijt} + \gamma_{ijt} G_{ijt} + \theta_{ijt} P_{ijt} + \delta_{ijt} L_{ijt}$ : les effets deviennent spécifiques à chaque journal $j$.

In [ ]:
cutoff = pd.Period('2017-06', freq='M')

models = {}

for period_label, period_filter in {
    'pre2017': model_data["month"] < cutoff,
    'post2017': model_data["month"] >= cutoff
}.items():
    
    models[period_label] = {}
    period_data = model_data[period_filter]
    
    for alignment in period_data["political_alignment"].unique():
        models[period_label][alignment] = {}
        subset = period_data[period_data["political_alignment"] == alignment]
        
        for journal in period_data["journal"].unique():
            sub_subset = subset[subset["journal"] == journal]

            if len(subset) >= 3:
                X = sub_subset[["na_share", "government", "pres_votes_share", "leg_votes_share"]]
                y = sub_subset[outcome]            
                model = sm.OLS(y, X).fit(cov_type='HC3')
                y_pred = model.predict(X)
                mape = mean_ape(y, y_pred)
                rmspe = root_mse(y, y_pred) / y.mean()

                models[period_label][alignment][journal] = {
                    "params": model.params.rename("params").to_frame(),
                    "pvalues": model.pvalues.rename("pval").to_frame(),
                    "mape": mape,
                    "rmspe": rmspe}

            else:
                models[period_label][alignment][journal] = {
                    "params": None,
                    "pvalues": None,
                    "mape": None,
                    "rmspe": None}

In [ ]:
summary = []

for period, alignment_dict in models.items():
    for alignment, journal_dict in alignment_dict.items():
        for journal, model_info in journal_dict.items():
            summary.append({
                "period": period,
                "alignment": alignment,
                "journal": journal,
                "mape": model_info["mape"],
                "rmspe": model_info["rmspe"]
            })

results = pd.DataFrame(summary)
results["alignment"] = pd.Categorical(results["alignment"], categories=nuances_order, ordered=True)
results = results.pivot_table(index=["alignment", "journal"],
                              columns="period",
                              values=["mape", "rmspe"],
                              observed=False)
results.columns = results.columns.swaplevel(0, 1)
results = results.reindex(columns=["pre2017", "post2017"], level=0)
results = results.sort_index()
results.index.names = ["alignment", "Journal"]

format_cols = results.columns
results.style.format({col: "{:.3f}" for col in format_cols})

In [ ]:
results_list = []

for period, alignment_dict in models.items():
    for alignment, journal_dict in alignment_dict.items():
        for journal, model_info in journal_dict.items():
            df = model_info["params"].join(model_info["pvalues"])
            df["period"] = period
            df["alignment"] = alignment
            df["journal"] = journal
            df["variable"] = df.index
            results_list.append(df.reset_index(drop=True))

results = pd.concat(results_list, ignore_index=True)
results = results.dropna(subset=["pval"])
results["alignment"] = pd.Categorical(results["alignment"], categories=nuances_order, ordered=True)
results = results.pivot_table(index=["variable", "alignment", "journal"],
                              columns="period",
                              values=["params", "pval"],
                              observed=False)

results.columns = results.columns.swaplevel(0, 1)
results = results.reindex(columns=["pre2017", "post2017"], level=0)
results = results.sort_index()
results.index.names = ["Variable", "alignment", "Journal"]

results.style.format({col: "{:.4f}" for col in results.columns})

In [ ]:
combinations = [
    ('na_share', 'pre2017'),
    ('na_share', 'post2017'),
    ('government', 'pre2017'),
    ('government', 'post2017'),
    ('leg_votes_share', 'pre2017'),
    ('leg_votes_share', 'post2017'),
    ('pres_votes_share', 'pre2017'),
    ('pres_votes_share', 'post2017'),
]

titles = [
    'NA Seats Share – Pre 2017',
    'NA Seats Share – Post 2017',
    'Government – Pre 2017',
    'Government – Post 2017',
    'Leg. Vote Share – Pre 2017',
    'Leg. Vote Share – Post 2017',
    'Pres. Vote Share – Pre 2017',
    'Pres. Vote Share – Post 2017',
]

heatmap_data = []
row_counts = []

for var, period in combinations:
    idx = results.index
    mask = (
        (idx.get_level_values(0) == var) &
        (idx.get_level_values('Journal') != 'Médiapart')
    )
    sub = results[mask][(period, 'params')]
    
    sub.index = sub.index.droplevel(0)
    pivot = sub.unstack(level=1)
    heatmap_data.append(pivot)
    row_counts.append(pivot.shape[0])

row_limits = []
for r in range(4):
    d1 = heatmap_data[2*r]
    d2 = heatmap_data[2*r+1]
    combined = pd.concat([d1, d2]).values.flatten()
    vmin, vmax = np.nanmin(combined), np.nanmax(combined)
    row_limits.append((vmin, vmax))

fig = plt.figure(figsize=(12, 16))
gs = gridspec.GridSpec(
    4, 3,
    width_ratios=[1, 1, 0.05],
    height_ratios=[
        max(row_counts[0:2])/max(row_counts),
        max(row_counts[2:4])/max(row_counts),
        max(row_counts[4:6])/max(row_counts),
        max(row_counts[6:8])/max(row_counts)
    ],
    hspace=0.3,
    figure=fig
)

axes = [fig.add_subplot(gs[i, j]) for i in range(4) for j in range(2)]
cbar_axes = [fig.add_subplot(gs[i, 2]) for i in range(4)]

for idx, ax in enumerate(axes):
    row = idx // 2
    vmin, vmax = row_limits[row]
    sns.heatmap(
        heatmap_data[idx],
        ax=ax,
        cmap='coolwarm',
        center=0,
        annot=True,
        fmt=".2f",
        vmin=vmin,
        vmax=vmax,
        cbar=(idx % 2 == 1),
        cbar_ax=(cbar_axes[row] if idx % 2 == 1 else None),
        cbar_kws={'label': 'Effect size'}
    )

    ax.set_title(titles[idx])
    ax.set_xlabel('')
    if idx % 2 == 0:
        ax.set_ylabel('')
        ax.tick_params(axis='y', labelrotation=0)
    else:
        ax.set_ylabel('')
        ax.set_yticks([])
        ax.set_yticklabels([])

plt.show()

# Attempt at Beta Regression

In [ ]:
import statsmodels.api as sm
from statsmodels.othermod.betareg import BetaModel

X = model_data_no_journal[["na_share", "government", "pres_votes_share", "leg_votes_share"]]
X = sm.add_constant(X)
y = model_data_no_journal[outcome]

epsilon = 1e-5
y_transformed = y.clip(epsilon, 1 - epsilon)

model = BetaModel(endog=y_transformed, exog=X)
results = model.fit(cov_type='HC3')

y_pred = results.predict(X)
mape = mean_ape(y_transformed, y_pred)
rmspe = root_mse(y_transformed, y_pred) / y_transformed.mean()

params = results.params.rename("coef").to_frame()
pvalues = results.pvalues.rename("pval").to_frame()
print(pd.merge(params, pvalues, left_index=True, right_index=True))
print("")
print(f"MAPE: {100*mape:.2f}%")
print(f"RMSPE: {rmspe:.5f}")